In [2]:
pip install google.generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 427.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.3 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 455.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from neo4j import GraphDatabase
#from openai.embeddings_utils import get_embedding
from openai import OpenAI

"""
LoadEmbedding: call OpenAI embedding API to generate embeddings for each property of node in Neo4j
Version: 1.1
"""
OPENAI_KEY = 0 #Enter your openai key here
EMBEDDING_MODEL = 0 #Enter your embedding model here
NEO4J_URL = 0 #Enter your Neo4j URL here
NEO4J_USER = 0 #Enter your Neo4j user here
NEO4J_PASSWORD = 0 #Enter your Neo4j password here
NEO4J_DATABASE = 0 #Enter your Neo4j database here

def get_embedding(client, text, model):
    response = client.embeddings.create(
                    input=text,
                    model=model,
                )
    return response.data[0].embedding

def LoadEmbedding(label, property):
    driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USER, NEO4J_PASSWORD), database=NEO4J_DATABASE)
    openai_client = OpenAI (api_key = OPENAI_KEY)

    with driver.session() as session:
        # get chunks in document, together with their section titles
        result = session.run(f"MATCH (ch:{label}) -[:HAS_PARENT]-> (s:Section) RETURN id(ch) AS id, s.title + ' >> ' + ch.{property} AS text")
        # call OpenAI embedding API to generate embeddings for each property of node
        # for each node, update the embedding property
        count = 0
        for record in result:
            id = record["id"]
            text = record["text"]
            
            # For better performance, text can be batched
            embedding = get_embedding(openai_client, text, EMBEDDING_MODEL)
            
            # key property of Embedding node differentiates different embeddings
            cypher = "CREATE (e:Embedding) SET e.key=$key, e.value=$embedding"
            cypher = cypher + " WITH e MATCH (n) WHERE id(n) = $id CREATE (n) -[:HAS_EMBEDDING]-> (e)"
            session.run(cypher,key=property, embedding=embedding, id=id )
            count = count + 1

        session.close()
        
        print("Processed " + str(count) + " " + label + " nodes for property @" + property + ".")
        return count




In [45]:

LoadEmbedding("Chunk", "sentences")

LoadEmbedding("Table", "name")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 54, offset: 53} for query: "MATCH (ch:Chunk) -[:HAS_PARENT]-> (s:Section) RETURN id(ch) AS id, s.title + ' >> ' + ch.sentences AS text"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 80, offset: 79} for query: 'CREATE (e:Embedding) SET e.key=$key, e.value=$embedding WITH e MATCH (n) WHERE id(n) = $id CREATE (n) -[:HAS_EMBEDDING]-> (e)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

Processed 4663 Chunk nodes for property @sentences.


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 54, offset: 53} for query: "MATCH (ch:Table) -[:HAS_PARENT]-> (s:Section) RETURN id(ch) AS id, s.title + ' >> ' + ch.name AS text"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 80, offset: 79} for query: 'CREATE (e:Embedding) SET e.key=$key, e.value=$embedding WITH e MATCH (n) WHERE id(n) = $id CREATE (n) -[:HAS_EMBEDDING]-> (e)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotificat

Processed 263 Table nodes for property @name.


263

In [51]:
!python -m streamlit run '/Users/adityaraj/Aditya/IIIT/3rd Year/6th Sem/Cloud Computing/Project/genai-stack/cs_bot_papers.py'


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.0.1.245:8501

/Users/adityaraj/Aditya/IIIT/3rd Year/6th Sem/Cloud Computing/Project/genai-stack/cs_bot_papers.py:11: LangChainDeprecationWarning: Importing Neo4jGraph from langchain.graphs is deprecated. Please replace deprecated imports:

>> from langchain.graphs import Neo4jGraph

with new imports of:

>> from langchain_community.graphs import Neo4jGraph
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.graphs import Neo4jGraph
/Users/adityaraj/Aditya/IIIT/3rd Year/6th Sem/Cloud Computing/Project/genai-stack/cs_bot_papers.py:42: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To u